# Big data? 🤗 Datasets to the rescue!

In [1]:
import os

# 设置代理
os.environ['http_proxy'] = 'http://127.0.0.1:7893'
os.environ['https_proxy'] = 'http://127.0.0.1:7893'
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7893'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7893'
os.environ['no_proxy'] = '127.0.0.1,localhost'
os.environ['NO_PROXY'] = '127.0.0.1,localhost'

In [5]:
!pip install zstandard

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install requests

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [ ]:
import os
import requests
import ssl
from urllib3.util.ssl_ import create_urllib3_context

# 方案1：配置代理和SSL设置
def setup_proxy_and_ssl():
    # 设置代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7893'
    os.environ['https_proxy'] = 'http://127.0.0.1:7893'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7893'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7893'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    
    # 创建一个session并配置SSL
    session = requests.Session()
    
    # 配置代理
    proxies = {
        'http': 'http://127.0.0.1:7893',
        'https': 'http://127.0.0.1:7893'
    }
    session.proxies.update(proxies)
    
    # 禁用SSL验证（仅用于测试）
    session.verify = False
    
    # 禁用SSL警告
    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    
    return session

# 使用配置好的session
session = setup_proxy_and_ssl()

try:
    url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
    response = session.get(url, timeout=30)
    print(f"请求成功，状态码: {response.status_code}")
    print(f"响应内容: {response.json()}")
except Exception as e:
    print(f"请求失败: {e}")

请求成功，状态码: 200
响应内容: [{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7785', 'repository_url': 'https://api.github.com/repos/huggingface/datasets', 'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/labels{/name}', 'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/comments', 'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/events', 'html_url': 'https://github.com/huggingface/datasets/pull/7785', 'id': 3439897018, 'node_id': 'PR_kwDODunzps6pyTM_', 'number': 7785, 'title': 'Fix Audio docstring by removing unsupported mono argument', 'user': {'login': 'tanuj-rai', 'id': 84439872, 'node_id': 'MDQ6VXNlcjg0NDM5ODcy', 'avatar_url': 'https://avatars.githubusercontent.com/u/84439872?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/tanuj-rai', 'html_url': 'https://github.com/tanuj-rai', 'followers_url': 'https://api.github.com/users/tanuj-rai/followers', 'following_url': 'https://api.

In [ ]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7785',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7785',
  'id': 3439897018,
  'node_id': 'PR_kwDODunzps6pyTM_',
  'number': 7785,
  'title': 'Fix Audio docstring by removing unsupported mono argument',
  'user': {'login': 'tanuj-rai',
   'id': 84439872,
   'node_id': 'MDQ6VXNlcjg0NDM5ODcy',
   'avatar_url': 'https://avatars.githubusercontent.com/u/84439872?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/tanuj-rai',
   'html_url': 'https://github.com/tanuj-rai',
   'followers_url': 'https://api.github.com/users/tanuj-rai/followers',
   'follo

In [ ]:
GITHUB_TOKEN = "xxx"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [ ]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import requests  # 添加缺失的requests导入


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
    session=None,  # 添加session参数
    headers=None,  # 添加headers参数
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    # 使用传入的session或创建新的
    if session is None:
        session = requests.Session()
        # 如果没有传入session，使用之前配置的代理设置
        session = setup_proxy_and_ssl()
    
    # 如果没有传入headers，使用默认的（不需要token进行基本查询）
    if headers is None:
        headers = {}

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        try:
            # Query with state=all to get both open and closed issues
            query = f"issues?page={page}&per_page={per_page}&state=all"
            url = f"{base_url}/{owner}/{repo}/{query}"
            
            # 使用session发送请求
            response = session.get(url, headers=headers, timeout=30)
            
            # 检查响应状态
            if response.status_code == 200:
                issues_data = response.json()
                batch.extend(issues_data)
            elif response.status_code == 403:
                print(f"Rate limit exceeded. Status: {response.status_code}")
                print("Sleeping for one hour...")
                time.sleep(60 * 60 + 1)
                continue
            else:
                print(f"Request failed with status: {response.status_code}")
                print(f"Response: {response.text[:200]}...")
                break
                
        except Exception as e:
            print(f"Error fetching page {page}: {e}")
            break

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    
    if all_issues:
        df = pd.DataFrame.from_records(all_issues)
        output_file = f"{issues_path}/{repo}-issues.jsonl"
        df.to_json(output_file, orient="records", lines=True)
        print(f"Downloaded {len(all_issues)} issues for {repo}! Dataset stored at {output_file}")
        return df
    else:
        print("No issues were downloaded.")
        return None

In [ ]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Request failed with status: 422
Response: {"message":"Pagination with the page parameter is not supported for large datasets, please use cursor based pagination (after/before)","documentation_url":"https://docs.github.com/rest/issues/issues#l...
Downloaded 1100 issues for datasets! Dataset stored at ./datasets-issues.jsonl


,url,repository_url,labels_url,comments_url,events_url,html_url,id,node_id,number,title,...,draft,pull_request,body,closed_by,reactions,timeline_url,performed_via_github_app,state_reason,sub_issues_summary,issue_dependencies_summary
0,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7785,3439897018,PR_kwDODunzps6pyTM_,7785,Fix Audio docstring by removing unsupported mo...,...,False,{'url': 'https://api.github.com/repos/huggingf...,This PR fixes issue #7745.\r\n\r\nWho can revi...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,None,None,NaN,NaN
1,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7783,3430715779,PR_kwDODunzps6pT7pg,7783,Adapt and test huggingface_hub v1.0.0.rc0,...,True,{'url': 'https://api.github.com/repos/huggingf...,Test as part of https://github.com/huggingface...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,None,None,NaN,NaN
2,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7782,3430341875,PR_kwDODunzps6pSozj,7782,set dev version,...,False,{'url': 'https://api.github.com/repos/huggingf...,None,"{'login': 'lhoestq', 'id': 42851186, 'node_id'...",{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,None,None,NaN,NaN
3,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/7781,3430332841,PR_kwDODunzps6pSm0C,7781,release: 4.1.1,...,False,{'url': 'https://api.github.com/repos/huggingf...,None,"{'login': 'lhoestq', 'id': 42851186, 'node_id'...",{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,None,None,NaN,NaN
4,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/issues...,3429267259,I_kwDODunzps7MZnc7,7780,BIGPATENT dataset inaccessible (deprecated scr...,...,NaN,NaN,dataset: https://huggingface.co/datasets/North...,None,{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/repos/huggingface/datas...,None,None,"{'total': 0, 'completed': 0, 'percent_complete...","{'blocked_by': 0, 'total_blocked_by': 0, 'bloc..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datasets,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://api.github.com/repos/huggingface/datas...,https://github.com/huggingface/datasets/pull/6749,2202310116,PR_kwDODunzps5qeoSk,6749,Fix fsspec tqdm callback,...,False,{'url': 'https://api.github.com/repos/huggingf...,Following changes at https://github.com/fsspec...,"{'login': 'lhoestq', 'id': 42851186, 'node_id'...",{'url': 'https://api.github.com/repos/huggingf...,https://api.github.com/

In [ ]:
from datasets import load_dataset

issues_dataset = load_dataset("json", data_files="datasets-issues.jsonl", split="train")
issues_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'sub_issues_summary', 'issue_dependencies_summary'],
    num_rows: 1100
})

In [ ]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/7158
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7158', 'html_url': 'https://github.com/huggingface/datasets/pull/7158', 'diff_url': 'https://github.com/huggingface/datasets/pull/7158.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7158.patch', 'merged_at': None}

>> URL: https://github.com/huggingface/datasets/pull/7704
>> Pull request: {'url': 'https://api.github.com/repos/huggingface/datasets/pulls/7704', 'html_url': 'https://github.com/huggingface/datasets/pull/7704', 'diff_url': 'https://github.com/huggingface/datasets/pull/7704.diff', 'patch_url': 'https://github.com/huggingface/datasets/pull/7704.patch', 'merged_at': None}

>> URL: https://github.com/huggingface/datasets/issues/7430
>> Pull request: None



In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [ ]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = session.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

In [ ]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = session.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
issues_with_comments_dataset.push_to_hub("datasets-issues")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        :  67%|######6   | 1.20MB / 1.80MB            

CommitInfo(commit_url='https://huggingface.co/datasets/yiwenX/datasets-issues/commit/a43540bffe8758749c08c3a8902c2a676c091eed', commit_message='Upload dataset', commit_description='', oid='a43540bffe8758749c08c3a8902c2a676c091eed', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yiwenX/datasets-issues', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yiwenX/datasets-issues'), pr_revision=None, pr_num=None)

In [2]:
from datasets import load_dataset
remote_dataset = load_dataset("yiwenX/datasets-issues", split="train")
remote_dataset

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1100 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'draft', 'pull_request', 'body', 'closed_by', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'sub_issues_summary', 'issue_dependencies_summary', 'is_pull_request'],
    num_rows: 1100
})